In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow 
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras import layers, callbacks
from torch import nn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from xgboost import XGBRFRegressor
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime
data_1 = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
data_2 = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
target = data_1.iloc[:,data_1.columns.str.contains('target')]
y = target.copy()
x = data_1.drop(y.columns , axis = 1)

x.head()


In [ ]:
data_1['date_time'] = pd.to_datetime(data_1.date_time, format = "%Y-%m-%d %H:%M:%S")
data_1['season'] = data_1.date_time.apply(lambda x: x.month%12/3 + 1).astype(int)
data_1.season.value_counts()

In [ ]:
data_2['date_time'] = pd.to_datetime(data_2.date_time, format = "%Y-%m-%d %H:%M:%S")
data_2['season'] = data_2.date_time.apply(lambda x: x.month%12/3 + 1).astype(int)
data_2['day'] = data_2.date_time.apply(lambda x:x.weekday())
data_2.season.value_counts()

In [ ]:
data_1['day'] = data_1.date_time.apply(lambda x:x.weekday())
data_1.head()

In [ ]:
x = x.drop('date_time', axis = 1)
x['season'] = data_1.season
x['day'] = data_1.day
x['Nc'] = (data_1.absolute_humidity * 100 / data_1.relative_humidity)
data_1['nc'] = x.Nc
x.head()

In [ ]:
data_2 = data_2.drop('date_time', axis = 1)
data_2['Nc'] = (data_2.absolute_humidity * 100 / data_2.relative_humidity)
data_2.head()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize = (20, 15))
i = 0
for j in target.columns:
    sns.barplot(x = data_1.season, y = data_1[f"{j}"], ax =ax[0][i])
    sns.histplot(x = data_1.nc, y = data_1[f"{j}"], ax =ax[1][i])
    #ax[i].set_title(f"{j}")
    
    i += 1

In [ ]:
x.hist(figsize = (24,12))

In [ ]:
data_1.corr()[target.columns].plot.bar(figsize = (24,20))

In [ ]:
fig = plt.figure(figsize = (24,18))
sns.heatmap(data_1.corr(), linewidth = 0.09)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(x, target, test_size = 0.3, random_state = 1)

In [ ]:
rc = RobustScaler()
t_x = pd.DataFrame(rc.fit_transform(train_x))
v_x = pd.DataFrame(rc.fit_transform(val_x))
test = pd.DataFrame(rc.fit_transform(data_2))

In [ ]:
model_1 = RandomForestRegressor(n_estimators = 500)
model_1.fit(train_x, train_y)
pred = model_1.predict(val_x)
error = np.sqrt(mean_squared_log_error(val_y,pred))
print(error)


In [ ]:
import xgboost
from sklearn.multioutput import MultiOutputRegressor
model_ = xgboost.XGBRFRegressor(n_estimators = 500)
model_2 = MultiOutputRegressor(model_)
model_2.fit(train_x, train_y )
pred_2 = model_2.predict(val_x)
error_2 = np.sqrt(mean_squared_log_error(pred_2, val_y))
print(error_2)

In [ ]:
model3 = keras.Sequential([layers.BatchNormalization(),
    layers.InputLayer(input_shape = [train_x.shape[1]]),
    layers.Dense(128, kernel_initializer = 'normal', activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dense(128, kernel_initializer = 'normal', activation = 'relu'),
    layers.Dense(3, activation = 'linear')
])

In [ ]:
model3.compile( optimizer = 'adam',
             loss = 'msle')
call = callbacks.EarlyStopping( patience = 10, 
                              min_delta = 0.001,
                              restore_best_weights = True)

In [ ]:
h = model3.fit(train_x, train_y, validation_split = 0.2, 
               callbacks = [call], epochs = 1000, batch_size = 128, verbose = 0)

In [ ]:
out = pd.DataFrame(h.history)
out.plot(figsize = (12,8))
print(np.sqrt(out.loss.min()), np.sqrt(out.val_loss.min()))

In [ ]:
ex = model3.predict(val_x)
result = np.sqrt(mean_squared_log_error(abs(ex), abs(val_y)))
print(result)

In [ ]:
ans =pd.DataFrame(model3.predict(data_2))

In [ ]:
output = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

In [ ]:
ans.columns = y.columns

In [ ]:
for i in ans.columns:
    output[f"{i}"] = ans[f"{i}"]
output.head()

In [ ]:
output.to_csv('submission_5', index = False)